## Contents
* [1. <b> Data Cleaning </b>](#first-bullet)
* [1.1 Exclusions and outliers removal](#second-bullet)
* [1.2 Imputation](#third-bullet)
* [2. <b> Feature Engineering </b>](#fourth-bullet)
* [2.1 Target Encoding](#fifth-bullet)
* [2.2 Benford law variables](#sixth-bullet)
* [2.3 Other interesting variables](#seventh-bullet)
* [3 <b> Storing the variables </b>](#eigth-bullet)

## Introduction

This code cleans the data, imputes missing values and performs feature engineering.
In the end it divides the variables into 3 parts so that it is easier to process them during feature selection.

In [1]:
from timeit import default_timer as timer
import pandas as pd
import scipy.stats
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import time
from itertools import permutations
%matplotlib inline
start_time = timer()

In [2]:
print(f'Duration: {(timer() - start_time) / 60: 0.2f} minutes, or{(timer() - start_time): 0.0f} seconds')

Duration:  0.00 minutes, or 0 seconds


## Data Cleaning <a class="anchor" id="first-bullet"></a>

In [3]:
data = pd.read_csv('card transactions.csv')

In [4]:
data.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [5]:
data.shape

(96753, 10)

In [6]:
data['Date'] = pd.to_datetime(data['Date'])

In [7]:
data['Merch zip'] = data['Merch zip'].fillna(0).astype('int').astype('str').str.zfill(5)

In [8]:
data['Merch zip'].value_counts() #0000 is missing value

38118    11868
00000     4656
63103     1650
08701     1267
22202     1250
         ...  
15312        1
63111        1
85628        1
73102        1
80934        1
Name: Merch zip, Length: 4568, dtype: int64

In [9]:
data['zip3'] = data['Merch zip'].str.slice(start = -3,stop = 5)

In [10]:
data['zip3']

0        118
1        803
2        706
3        118
4        118
        ... 
96748    042
96749    248
96750    150
96751    656
96752    606
Name: zip3, Length: 96753, dtype: object

In [11]:
data['Merch zip'] = data['Merch zip'].replace('00000',np.nan)

In [12]:
data['zip3'] = data['zip3'].replace('000',np.nan)

### Exclusions and outliers removal <a class="anchor" id="second-bullet"></a>

In [13]:
#Remove all but the “P” transactions. 
data = data[data['Transtype']=='P']

In [14]:
#Single large transaction outlier
data['Amount'].value_counts().reset_index().sort_values(by = 'index')
data = data[data['Amount']<3000000]

In [15]:
data.shape

(96397, 11)

### Imputation <a class="anchor" id="third-bullet"></a>

#### merchnum

In [16]:
#If zero replace with nan
data['Merchnum'] = data['Merchnum'].replace('0',np.nan)
data['Merchnum'].value_counts(dropna = False)

930090121224     9310
NaN              3251
5509006296254    2131
9900020006406    1714
602608969534     1092
                 ... 
8000880818800       1
2200000003085       1
607960237330        1
9600063580068       1
912100650006        1
Name: Merchnum, Length: 13090, dtype: int64

In [17]:
#this is not the mode imputation logic. Check ZYMARK CORPORATION
# merchdes_merchnum = {}
# for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
#     if merchdes not in merchdes_merchnum :
#         merchdes_merchnum[merchdes] = data.loc[index, 'Merchnum']


In [18]:
merchdes_merchnum = data[~pd.isnull(data['Merchnum'])].groupby(['Merch description'])['Merchnum'].apply(lambda x: x.mode()[0])

In [19]:
merchdes_merchnum.value_counts()

930090121224     178
4060500620006    110
5509006296254     94
7284720084825     27
8664000639778     22
                ... 
935049353222       1
6070081910003      1
490901561229       1
5000001134         1
912100650006       1
Name: Merchnum, Length: 11697, dtype: int64

In [20]:
data.loc[201]

Recnum                                    202
Cardnum                            5142140436
Date                      2006-01-03 00:00:00
Merchnum                                  NaN
Merch description    MONTGOMERY COLLEGE-PHONE
Merch state                                MD
Merch zip                               20850
Transtype                                   P
Amount                                  266.0
Fraud                                       0
zip3                                      850
Name: 201, dtype: object

In [21]:
data[data['Merch description'] == 'MONTGOMERY COLLEGE-PHONE']

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,zip3
201,202,5142140436,2006-01-03,NaN,MONTGOMERY COLLEGE-PHONE,MD,20850,P,266.00,0,850
39516,39517,5142141115,2006-05-26,NaN,MONTGOMERY COLLEGE-PHONE,MD,20850,P,58.45,0,850
64192,64193,5142141115,2006-08-17,NaN,MONTGOMERY COLLEGE-PHONE,MD,20850,P,278.00,0,850
94748,94749,5142140436,2006-12-20,NaN,MONTGOMERY COLLEGE-PHONE,MD,20850,P,25.00,0,850


In [22]:
# fill in by mapping with Merch description
data['Merchnum'] = data['Merchnum'].fillna(data['Merch description'].map(merchdes_merchnum))


In [23]:
# assign unknown for adjustments transactions
data['Merchnum'] = data['Merchnum'].mask(data['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merchnum'] = data['Merchnum'].mask(data['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')

#### merch state

In [24]:
# dict for mapping
# zip_state = {}
# for index, zip5 in data[data['Merch zip'].notnull()]['Merch zip'].items():
#     if zip5 not in zip_state:
#         zip_state[zip5] = data.loc[index, 'Merch state']

zip_state = data[~pd.isnull(data['Merch state'])].groupby(['Merch zip'])['Merch state'].apply(lambda x: x.mode()[0])


In [25]:
zip_state

Merch zip
00001    PA
00002    MD
00003    GA
00005    IA
00006    IL
         ..
99755    AK
99801    AK
99835    AK
99901    AK
99999    CA
Name: Merch state, Length: 4538, dtype: object

In [26]:
zip_state['00926'] = 'PR'
zip_state['00929'] = 'PR'
zip_state['00934'] = 'PR'
zip_state['00902'] = 'PR'
zip_state['00738'] = 'PR'
zip_state['90805'] = 'CA'
zip_state['76302'] = 'TX'
zip_state['00914'] = 'PR'
zip_state['95461'] = 'CA'
zip_state['00680'] = 'PR'
zip_state['00623'] = 'PR'
zip_state['00726'] = 'PR'
zip_state['00936'] = 'PR'
zip_state['12108'] = 'NY'
zip_state['00791'] = 'PR'
zip_state['00907'] = 'PR'
zip_state['00922'] = 'PR'
zip_state['00920'] = 'PR'
zip_state['00801'] = 'VI'
zip_state['31040'] = 'GA'
zip_state['41160'] = 'KY'
zip_state['00681'] = 'PR'


In [27]:
merchnum_state = data[~pd.isnull(data['Merch state'])].groupby(['Merchnum'])['Merch state'].apply(lambda x: x.mode()[0])

In [28]:
merchdes_state = data[~pd.isnull(data['Merch state'])].groupby(['Merch description'])['Merch state'].apply(lambda x: x.mode()[0])

In [29]:
# merchnum_state = {}
# for index, merchnum in data[data['Merchnum'].notnull()]['Merchnum'].items():
#     if merchnum not in merchnum_state :
#         merchnum_state[merchnum] = data.loc[index, 'Merch state']

# merchdes_state = {}
# for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
#     if merchdes not in merchdes_state :
#         merchdes_state[merchdes] = data.loc[index, 'Merch state']


In [30]:
# fill in by mapping with zip, merchnum and merch description
data['Merch state'] = data['Merch state'].fillna(data['Merch zip'].map(zip_state))
data['Merch state'] = data['Merch state'].fillna(data['Merchnum'].map(merchnum_state))
data['Merch state'] = data['Merch state'].fillna(data['Merch description'].map(merchdes_state))


In [31]:
# assign unknown for adjustments transactions
data['Merch state'] = data['Merch state'].mask(data['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merch state'] = data['Merch state'].mask(data['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')


#### merch zip

In [32]:
data['Merch zip'] = data['Merch zip'].replace('00000',np.nan)

In [33]:
data['Merch zip'].value_counts(dropna = False)

38118    11868
NaN       4300
63103     1650
08701     1267
22202     1250
         ...  
66801        1
35487        1
48151        1
55446        1
66216        1
Name: Merch zip, Length: 4568, dtype: int64

In [34]:
merchnum_zip = data[~pd.isnull(data['Merch zip'])].groupby(['Merchnum'])['Merch zip'].apply(lambda x: x.mode()[0])

In [35]:
merchnum_zip

Merchnum
003100006NOT6    41011
004740006ABC6    82520
005590006PNB6    80439
014430619  14    90502
014938913  51    84116
                 ...  
DU49038320006    19547
JCPENNE9 CO      19454
PENNE9 CO #05    43623
PENNE9 CO #68    40422
unknown          00007
Name: Merch zip, Length: 12834, dtype: object

In [36]:
# merchnum_zip = {}
# for index, merchnum in data[data['Merchnum'].notnull()]['Merchnum'].items():
#     if merchnum not in merchnum_zip :
#         merchnum_zip [merchnum] = data.loc[index, 'Merch zip']


In [37]:
merchdes_zip = data[~pd.isnull(data['Merch zip'])].groupby(['Merch description'])['Merch zip'].apply(lambda x: x.mode()[0])

In [38]:
# merchdes_zip = {}
# for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
#     if merchdes not in merchdes_zip :
#         merchdes_zip [merchdes] = data.loc[index, 'Merch zip']


In [39]:
# fill in by mapping with merchnum and merch description
data['Merch zip'] = data['Merch zip'].fillna(data['Merchnum'].map(merchnum_zip))
data['Merch zip'] = data['Merch zip'].fillna(data['Merch description'].map(merchdes_zip))


In [40]:
# assign unknown for adjustments transactions
data['Merch zip'] = data['Merch zip'].mask(data['Merch zip'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merch zip'] = data['Merch zip'].mask(data['Merch zip'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')

In [41]:
data['Merch zip'].value_counts(dropna = False)

38118    11868
22202     2172
NaN       1723
63103     1650
08701     1269
         ...  
84118        1
89801        1
18508        1
91248        1
33413        1
Name: Merch zip, Length: 4568, dtype: int64

In [42]:
# fill the rest with unknown
data = data.fillna('unknown')
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96752
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Recnum             96397 non-null  int64         
 1   Cardnum            96397 non-null  int64         
 2   Date               96397 non-null  datetime64[ns]
 3   Merchnum           96397 non-null  object        
 4   Merch description  96397 non-null  object        
 5   Merch state        96397 non-null  object        
 6   Merch zip          96397 non-null  object        
 7   Transtype          96397 non-null  object        
 8   Amount             96397 non-null  float64       
 9   Fraud              96397 non-null  int64         
 10  zip3               96397 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 10.8+ MB


In [43]:
# data['zip3'] = data['Merch zip'].str.slice(start = -3,stop = 5)

In [44]:
data['Cardnum'] = data['Cardnum'].astype('str')

In [45]:
# data['Merch description'].value_counts().tail(40)
# [i for i in data['Merch description'] if i in ['jewelry','electronics','online']]
# #recurring charges, same amount
# x = data.groupby(['Merchnum','Amount']).Recnum.count()
# x[x>100]

## Feature Engineering <a class="anchor" id="fourth-bullet"></a>

In [46]:
data['Cardnum_Merchnum'] = data['Cardnum'].astype('str') + data['Merchnum'].astype('str')
data['Cardnum_Zip'] = data['Cardnum'].astype('str') + data['Merch zip']
data['Cardnum_state'] = data['Cardnum'] + data['Merch state']
data['Merchnum_Zip'] = data['Merchnum'] + data['Merch zip']

In [47]:
data['Cardnum_Merchnum_state'] = data['Cardnum'] + data['Merchnum'] + data['Merch state']
data['Cardnum_Merchnum_zip'] = data['Cardnum'] + data['Merchnum'] + data['Merch zip']

In [48]:
data['Cardnum_Zip3'] = data['Cardnum'].astype('str') + data['zip3']
data['Merchnum_Zip3'] = data['Merchnum'].astype('str') + data['zip3']
data['Cardnum_Merchnum_zip3'] = data['Cardnum'].astype('str') + data['Merchnum'].astype('str') + data['zip3']

In [49]:
entities = ['Cardnum','Merchnum','Cardnum_Merchnum','Cardnum_Zip','Cardnum_state','Merchnum_Zip','Cardnum_Merchnum_state','Cardnum_Merchnum_zip',
           'Cardnum_Zip3','Merchnum_Zip3','Cardnum_Merchnum_zip3']

In [50]:
data1=data.copy()
final=data.copy()
data1['check_date']=data1.Date
data1['check_record']=data1.Recnum
final.shape

(96397, 20)

### Day-since, frequency and amount variables

In [51]:
start = timer()
for entity in entities:    
    try: print('Run time for the last entity ----------------- {}s'.format(timeit.default_timer() - st))
    except: print('')
    st = timer()    
    ## Day-since variables:   
    df_l = data1[['Recnum', 'Date', entity]]
    df_r = data1[['check_record', 'check_date', entity, 'Amount']]    
    temp = pd.merge(df_l, df_r, left_on = entity, right_on = entity)   
    temp1 = temp[temp.Recnum > temp.check_record][['Recnum','Date','check_date']]\
                                                   .groupby('Recnum')[['Date', 'check_date']].last()
    mapper = (temp1.Date - temp1.check_date).dt.days
    final[entity + '_day_since'] = final.Recnum.map(mapper)
#     final[entity + '_day_since'].fillna((final.Date - pd.to_datetime('2006-01-01')).dt.days, inplace = True)
    #using default values as 365 
    final[entity + '_day_since'].fillna(365, inplace = True)
    print('\n' + entity + '_day_since ---> Done')   
    ## Frequency & Amount variables:   
    for time in [0,1,3,7,14,30]:       
        temp2 = temp[(temp.check_date >= (temp.Date - dt.timedelta(time))) &\
                       (temp.Recnum >= temp.check_record)][['Recnum', entity, 'Amount']]       
        col_name = entity + '_count_' + str(time)    
        mapper2 = temp2.groupby('Recnum')[entity].count()      
        final[col_name] = final.Recnum.map(mapper2)        
        print(col_name + ' ---> Done')       
        final[entity + '_avg_' + str(time)] = final.Recnum.map(temp2.groupby('Recnum')['Amount'].mean())
        final[entity + '_max_' + str(time)] = final.Recnum.map(temp2.groupby('Recnum')['Amount'].max())
        final[entity + '_med_' + str(time)] = final.Recnum.map(temp2.groupby('Recnum')['Amount'].median())
        final[entity + '_total_' + str(time)] = final.Recnum.map(temp2.groupby('Recnum')['Amount'].sum())
        final[entity + '_actual/avg_' + str(time)] = final['Amount'] / final[entity + '_avg_' + str(time)]
        final[entity + '_actual/max_' + str(time)] = final['Amount'] / final[entity + '_max_' + str(time)]
        final[entity + '_actual/med_' + str(time)] = final['Amount'] / final[entity + '_med_' + str(time)]
        final[entity + '_actual/toal_' + str(time)] = final['Amount'] / final[entity + '_total_' + str(time)]       
        print(entity + ' amount variables over past ' + str(time) + ' ---> Done')

print('Total run time: {}mins'.format((timer() - start)/60))




Cardnum_day_since ---> Done
Cardnum_count_0 ---> Done
Cardnum amount variables over past 0 ---> Done
Cardnum_count_1 ---> Done
Cardnum amount variables over past 1 ---> Done
Cardnum_count_3 ---> Done
Cardnum amount variables over past 3 ---> Done
Cardnum_count_7 ---> Done
Cardnum amount variables over past 7 ---> Done
Cardnum_count_14 ---> Done
Cardnum amount variables over past 14 ---> Done
Cardnum_count_30 ---> Done
Cardnum amount variables over past 30 ---> Done


Merchnum_day_since ---> Done
Merchnum_count_0 ---> Done
Merchnum amount variables over past 0 ---> Done
Merchnum_count_1 ---> Done
Merchnum amount variables over past 1 ---> Done
Merchnum_count_3 ---> Done
Merchnum amount variables over past 3 ---> Done
Merchnum_count_7 ---> Done
Merchnum amount variables over past 7 ---> Done
Merchnum_count_14 ---> Done
Merchnum amount variables over past 14 ---> Done
Merchnum_count_30 ---> Done
Merchnum amount variables over past 30 ---> Done


Cardnum_Merchnum_day_since ---> Done
Card

In [52]:
len(final.columns)

625

In [53]:
start = timer()
# velocity change variables
for ent in entities:
    for d in ['0', '1']:
        for dd in ['3','7', '14', '30']:
            final[ent + '_count_' + d + '_by_' + dd] =\
            final[ent + '_count_' + d]/(final[ent + '_count_' + dd])/float(dd)
            
print('run time: {}s'.format(timer() - start))


run time: 1.0801476999999977s


In [54]:
len(final.columns)

713

### Target Encoded variable <a class="anchor" id="fifth-bullet"></a>

In [55]:
#remove OOT data

oot_date = dt.datetime(2006,11,1)

In [56]:
data['dow'] = data['Date'].apply(lambda x : calendar.day_name[x.weekday()])
train_test = data[data['Date']<oot_date]
len(train_test)


83970

In [57]:
len(data) - len(train_test)

12427

In [58]:
#smoothing
c = 4; nmid = 20; y_avg = train_test['Fraud'].mean()
y_dow = train_test.groupby('dow')['Fraud'].mean()
num = train_test.groupby('dow').size()
y_dow
train_test.groupby('dow').size()
y_avg
y_dow

dow
Friday       0.009788
Monday       0.010095
Saturday     0.018626
Sunday       0.025994
Thursday     0.007127
Tuesday      0.009630
Wednesday    0.008711
Name: Fraud, dtype: float64

In [59]:
y_dow_smooth = y_avg + (y_dow - y_avg)/( 1 + np.exp(-(num - nmid)/c))
y_dow_smooth

dow
Friday       0.009788
Monday       0.010095
Saturday     0.018626
Sunday       0.025994
Thursday     0.007127
Tuesday      0.009630
Wednesday    0.008711
dtype: float64

In [60]:
data['dow_risk'] = data['dow'].map(y_dow_smooth)

In [61]:
y_dow = y_dow.reset_index()
cats = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
y_dow['dow'] = pd.Categorical(y_dow['dow'],categories=cats,ordered = True)
y_dow = y_dow.sort_values('dow').set_index('dow')
y_dow

,Fraud
dow,
Monday,0.010095
Tuesday,0.009630
Wednesday,0.008711
Thursday,0.007127
Friday,0.009788
Saturday,0.018626
Sunday,0.025994


### Benford's Variables <a class="anchor" id="sixth-bullet"></a>

In [62]:
data_benford = data[data['Merch description'].str.match(r"[Ff][Ee][Dd][Ee][Xx]")==False].copy()
data_benford['Amount'] = data_benford['Amount'].astype('str')

In [63]:
def n_low(df):
    return ((df['Amount'].str.extract(r"([1-9])")=='1')|(df['Amount'].str.extract(r"([1-9])")=='2')).sum()

In [64]:
# cardnums = 
cardnums = data_benford.groupby('Cardnum').apply(n_low)
cardnums['n'] = data_benford.groupby('Cardnum')['Amount'].count()
cardnums['n_low'] = data_benford.groupby('Cardnum').apply(n_low)
cardnums

,0,n,n_low
Cardnum,,,
5142110002,1,1,1
5142110081,0,4,0
5142110313,2,3,2
5142110402,3,11,3
5142110434,1,1,1
...,...,...,...
5142310397,0,1,0
5142310525,1,4,1
5142310598,2,2,2


In [65]:
cardnums['n_high'] = cardnums['n'] - cardnums['n_low'] 
cardnums = cardnums[['n','n_low','n_high']]
cardnums['n_low'] = cardnums['n_low'].replace(0,1)
cardnums['n_high'] = cardnums['n_high'].replace(0,1)

<ipython-input-65-f4d7fd64be10>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardnums['n_low'] = cardnums['n_low'].replace(0,1)
<ipython-input-65-f4d7fd64be10>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cardnums['n_high'] = cardnums['n_high'].replace(0,1)


In [66]:
cardnums['R'] = 1.096* cardnums['n_low']/cardnums['n_high']
cardnums['1/R'] = 1/cardnums['R']
cardnums['U'] = cardnums[['R','1/R']].max(axis = 1)
cardnums['t'] = (cardnums['n'] - 15)/3
cardnums['U*_card'] = 1 + ((cardnums['U'] - 1)/(1+np.exp(-cardnums['t'])))
# cardnums.sort_values('U*',ascending = False)

In [67]:
# merchnums = 
merchnums = data_benford.groupby('Merchnum').apply(n_low)
merchnums['n'] = data_benford.groupby('Merchnum')['Amount'].count()
merchnums['n_low'] = data_benford.groupby('Merchnum').apply(n_low)
merchnums
merchnums['n_high'] = merchnums['n'] - merchnums['n_low'] 
merchnums = merchnums[['n','n_low','n_high']]
merchnums['n_low'] = merchnums['n_low'].replace(0,1)
merchnums['n_high'] = merchnums['n_high'].replace(0,1)
merchnums['R'] = 1.096* merchnums['n_low']/merchnums['n_high']
merchnums['1/R'] = 1/merchnums['R']
merchnums['U'] = merchnums[['R','1/R']].max(axis = 1)
merchnums['t'] = (merchnums['n'] - 15)/3
merchnums['U*_merch'] = 1 + ((merchnums['U'] - 1)/(1+np.exp(-merchnums['t'])))
# merchnums.sort_values('U*',ascending = False)

In [68]:
cardnums = cardnums.reset_index()
merchnums = merchnums.reset_index()


In [69]:
final = final.merge(cardnums[['Cardnum','U*_card']],on = ['Cardnum'], how = 'left')

In [70]:
final.columns

Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud',
       ...
       'Merchnum_Zip3_count_1_by_30', 'Cardnum_Merchnum_zip3_count_0_by_3',
       'Cardnum_Merchnum_zip3_count_0_by_7',
       'Cardnum_Merchnum_zip3_count_0_by_14',
       'Cardnum_Merchnum_zip3_count_0_by_30',
       'Cardnum_Merchnum_zip3_count_1_by_3',
       'Cardnum_Merchnum_zip3_count_1_by_7',
       'Cardnum_Merchnum_zip3_count_1_by_14',
       'Cardnum_Merchnum_zip3_count_1_by_30', 'U*_card'],
      dtype='object', length=714)

In [71]:
final = final.merge(merchnums[['Merchnum','U*_merch']],on = ['Merchnum'], how = 'left')

In [72]:
# final.shape

In [73]:
### Indicator variables: These variables look in the future

In [74]:
# begin = timer()
# attributes = ['Cardnum','Merchnum']
# for i in attributes:
#     for v in ['3','7','30']:
#         st=timer()
#         mapper=final.groupby(i)[i + '_count_'+v].max()
#         final['max_count_by_'+i+"_"+v] = final[i].map(mapper)
#         print(f'max_count_by_{i}_{v}----->Done')
#     print(f'run time for entity {i}----->{timer()-st}s')
# print(f'Total run time:{(timer()-begin)/60}min')
        
        

### Interesting variables <a class="anchor" id="seventh-bullet"></a>

In [75]:
# Recurring charges with same merchant and same amount 
data1['merch_amount'] = data['Merchnum'] + data['Amount'].astype('str')
entities = ['merch_amount']
start = timer()
for entity in entities:    
    try: print('Run time for the last entity ----------------- {}s'.format(timeit.default_timer() - st))
    except: print('')
    st = timer()    
    ## Day-since variables:   
    df_l = data1[['Recnum', 'Date', entity]]
    df_r = data1[['check_record', 'check_date', entity, 'Amount']]    
    temp = pd.merge(df_l, df_r, left_on = entity, right_on = entity)   
    temp1 = temp[temp.Recnum > temp.check_record][['Recnum','Date','check_date']]\
                                                   .groupby('Recnum')[['Date', 'check_date']].last()
    mapper = (temp1.Date - temp1.check_date).dt.days
    final[entity + '_day_since'] = final.Recnum.map(mapper)
    final[entity + '_day_since'].fillna((final.Date - pd.to_datetime('2010-01-01')).dt.days, inplace = True)
    print('\n' + entity + '_day_since ---> Done')   
    ## Frequency variables:   
    for time in [0,1,3,7,14,30]:       
        temp2 = temp[(temp.check_date >= (temp.Date - dt.timedelta(time))) &\
                       (temp.Recnum >= temp.check_record)][['Recnum', entity, 'Amount']]       
        col_name = entity + '_count_' + str(time)    
        mapper2 = temp2.groupby('Recnum')[entity].count()      
        final[col_name] = final.Recnum.map(mapper2)        
        print(col_name + ' ---> Done')       
        

print('Total run time: {}mins'.format((timer() - start)/60))



merch_amount_day_since ---> Done
merch_amount_count_0 ---> Done
merch_amount_count_1 ---> Done
merch_amount_count_3 ---> Done
merch_amount_count_7 ---> Done
merch_amount_count_14 ---> Done
merch_amount_count_30 ---> Done
Total run time: 0.2508825833333333mins


In [76]:
start = timer()
# velocity change variables
for ent in entities:
    for d in ['0', '1']:
        for dd in ['7', '14', '30']:
            final[ent + '_count_' + d + '_by_' + dd] =\
            final[ent + '_count_' + d]/(final[ent + '_count_' + dd])/float(dd)
            
print('run time: {}s'.format(timer() - start))

run time: 0.02593410000008589s


In [77]:
data.columns


Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud', 'zip3',
       'Cardnum_Merchnum', 'Cardnum_Zip', 'Cardnum_state', 'Merchnum_Zip',
       'Cardnum_Merchnum_state', 'Cardnum_Merchnum_zip', 'Cardnum_Zip3',
       'Merchnum_Zip3', 'Cardnum_Merchnum_zip3', 'dow', 'dow_risk'],
      dtype='object')

In [78]:
#for a particular card, how many unique states and merchants till that point
#for a particular card, difference between the time it was used for a merchant in a different state. If same state in previous transaction then 0

In [79]:
final['Previous state'] = final.groupby(['Cardnum'])['Merch state'].shift()

In [80]:
final['Cardnum_days_since_diff_state'] = np.where(final['Previous state']!=final['Merch state'],final['Cardnum_day_since'],0)

In [81]:
final['Cardnum_days_since_diff_state'].value_counts()

0.0      51014
1.0      13551
2.0       5646
3.0       4217
4.0       3267
         ...  
233.0        1
180.0        1
242.0        1
214.0        1
235.0        1
Name: Cardnum_days_since_diff_state, Length: 210, dtype: int64

In [82]:
final.Cardnum_day_since.corr(final.Cardnum_days_since_diff_state)

0.9904452111287436

In [83]:
final['Cardnum_days_since_diff_state'].value_counts()

0.0      51014
1.0      13551
2.0       5646
3.0       4217
4.0       3267
         ...  
233.0        1
180.0        1
242.0        1
214.0        1
235.0        1
Name: Cardnum_days_since_diff_state, Length: 210, dtype: int64

In [84]:
final = final.drop(['Previous state'],axis = 1)

In [85]:
data.columns

Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud', 'zip3',
       'Cardnum_Merchnum', 'Cardnum_Zip', 'Cardnum_state', 'Merchnum_Zip',
       'Cardnum_Merchnum_state', 'Cardnum_Merchnum_zip', 'Cardnum_Zip3',
       'Merchnum_Zip3', 'Cardnum_Merchnum_zip3', 'dow', 'dow_risk'],
      dtype='object')

In [86]:

begin = timer()
count_count_entities_vars = 0
combDict = {'comb3':[('Cardnum','Merchnum'),('Merchnum','Merch state')]}

for (entity,field) in combDict['comb3']:
    st = timer()
    df_c = data1[['Recnum','Date',entity]]
    df_d = data1[['check_record','check_date',entity,field]]
    temp = pd.merge(df_c,df_d,on = entity)
    for offset_t in [1,3,7,14,30,60]:
        count_dat_df = temp[(temp.check_date >= temp.Date - dt.timedelta(offset_t)) & (temp.Recnum >=temp.check_record)]
        col_name = f'{entity}_unique_count_for_{field}_{offset_t}'
        mapper = count_dat_df.groupby(['Recnum'])[field].nunique()
        final[col_name] = final.Recnum.map(mapper)
        count_count_entities_vars = count_count_entities_vars + 1
    
    print(f'Run time for entity {entity} in field {field}-----------------{timer() - st}s')

print(f'Total run time{(timer()-begin)/60} min')
print(f'Total count by entities variables {count_count_entities_vars}')

Run time for entity Cardnum in field Merchnum-----------------20.261359800000037s
Run time for entity Merchnum in field Merch state-----------------117.73758169999996s
Total run time2.3009789299999985 min
Total count by entities variables 12


In [87]:
len(final.columns)

741

In [101]:

begin = timer()
count_count_entities_vars = 0
combDict = {'comb3':[('Merchnum','Cardnum'),('Merch zip','Merchnum')]}

for (entity,field) in combDict['comb3']:
    st = timer()
    df_c = data1[['Recnum','Date',entity]]
    df_d = data1[['check_record','check_date',entity,field]]
    temp = pd.merge(df_c,df_d,on = entity)
    for offset_t in [1,3,7,14,30,60]:
        count_dat_df = temp[(temp.check_date >= temp.Date - dt.timedelta(offset_t)) & (temp.Recnum >=temp.check_record)]
        col_name = f'{entity}_unique_count_for_{field}_{offset_t}'
        mapper = count_dat_df.groupby(['Recnum'])[field].nunique()
        final[col_name] = final.Recnum.map(mapper)
        count_count_entities_vars = count_count_entities_vars + 1
    
    print(f'Run time for entity {entity} in field {field}-----------------{timer() - st}s')

print(f'Total run time{(timer()-begin)/60} min')
print(f'Total count by entities variables {count_count_entities_vars}')

Run time for entity Merchnum in field Cardnum-----------------203.25116800000023s
Run time for entity Merch zip in field Merchnum-----------------278.79601170000024s
Total run time8.035708196666663 min
Total count by entities variables 12


In [102]:

begin = timer()
count_count_entities_vars = 0
combDict = {'comb3':[('Cardnum','zip3'),('zip3','Cardnum'),('zip3','Merchnum'),('Merchnum','zip3')]}

for (entity,field) in combDict['comb3']:
    st = timer()
    df_c = data1[['Recnum','Date',entity]]
    df_d = data1[['check_record','check_date',entity,field]]
    temp = pd.merge(df_c,df_d,on = entity)
    for offset_t in [1,3,7,14,30,60]:
        count_dat_df = temp[(temp.check_date >= temp.Date - dt.timedelta(offset_t)) & (temp.Recnum >=temp.check_record)]
        col_name = f'{entity}_unique_count_for_{field}_{offset_t}'
        mapper = count_dat_df.groupby(['Recnum'])[field].nunique()
        final[col_name] = final.Recnum.map(mapper)
        count_count_entities_vars = count_count_entities_vars + 1
    
    print(f'Run time for entity {entity} in field {field}-----------------{timer() - st}s')

print(f'Total run time{(timer()-begin)/60} min')
print(f'Total count by entities variables {count_count_entities_vars}')

Run time for entity Cardnum in field zip3-----------------30.925923699999657s
Run time for entity zip3 in field Cardnum-----------------498.5713573000003s
Run time for entity zip3 in field Merchnum-----------------419.65203399999973s
Run time for entity Merchnum in field zip3-----------------137.90960069999983s
Total run time18.12005050833334 min
Total count by entities variables 24


In [103]:
#Fill values for benford fed ex
final['U*_merch'] = final['U*_merch'].fillna(np.mean(final['U*_merch']))
final['U*_card'] = final['U*_card'].fillna(np.mean(final['U*_card']))

In [104]:
final['dow'] = final['Date'].apply(lambda x : calendar.day_name[x.weekday()])
final['dow_risk'] = final['dow'].map(y_dow_smooth)

In [105]:
final.columns

Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud',
       ...
       'zip3_unique_count_for_Merchnum_30',
       'zip3_unique_count_for_Merchnum_60', 'Merchnum_unique_count_for_zip3_1',
       'Merchnum_unique_count_for_zip3_3', 'Merchnum_unique_count_for_zip3_7',
       'Merchnum_unique_count_for_zip3_14',
       'Merchnum_unique_count_for_zip3_30',
       'Merchnum_unique_count_for_zip3_60', 'dow', 'dow_risk'],
      dtype='object', length=779)

## Storing the variables <a class="anchor" id="eigth-bullet"></a>

In [106]:
data.columns

Index(['Recnum', 'Cardnum', 'Date', 'Merchnum', 'Merch description',
       'Merch state', 'Merch zip', 'Transtype', 'Amount', 'Fraud', 'zip3',
       'Cardnum_Merchnum', 'Cardnum_Zip', 'Cardnum_state', 'Merchnum_Zip',
       'Cardnum_Merchnum_state', 'Cardnum_Merchnum_zip', 'Cardnum_Zip3',
       'Merchnum_Zip3', 'Cardnum_Merchnum_zip3', 'dow', 'dow_risk'],
      dtype='object')

In [107]:
final_columns = set(final.columns) - set(['Cardnum','Date','Merchnum','Merch description','Merch state','Merch zip','Transtype','dow','zip3','Cardnum_Merchnum','Cardnum_Zip','Cardnum_state','Merchnum_Zip','Cardnum_Merchnum_state','Cardnum_Merchnum_zip','Cardnum_Zip3','Merchnum_Zip3','Cardnum_Merchnum_zip3'])
merch_columns = [i for i in final_columns if 'Merch' in i[0:10]]
non_merch_columns = [i for i in final_columns if i not in merch_columns]


In [108]:
if 'Fraud' not in non_merch_columns[0:150]:
    first_half = non_merch_columns[0:150] + ['Fraud']
else:
    first_half = non_merch_columns[0:150]
    
if 'Fraud' not in non_merch_columns[150:]:
    second_half = non_merch_columns[150:] + ['Fraud']
else:
    second_half = non_merch_columns[150:]
    
if 'Recnum' not in second_half:
    second_half = second_half + ['Recnum']
    
if 'Recnum' not in first_half:
    first_half = first_half + ['Recnum']


In [109]:
final3 = final.filter(merch_columns+['Recnum','Fraud'])
final1 = final.filter(first_half)
final2 = final.filter(second_half)

In [ ]:
# len(final3.columns) + len(final2.columns) + len(final1.columns) - 6

In [ ]:
# len(set(list(final3.columns) + list(final2.columns) + list(final1.columns)))

In [ ]:
# len(final_columns)

In [110]:
final1.to_csv("final1_1.csv")
final2.to_csv("final2_2.csv")
final3.to_csv("final3_3.csv")